
# Projeto 2 - SCC0252 (Visualização Computacional)


*   Otávio Ferracioli Coletti - 11767796
*   Fernando César Lopes Barbosa Filho - 10260559

\\

---

In [1]:
# Bibliotecas importadas
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import Button, Layout
from matplotlib.spines import rcParams
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib import legend
from pandas import Series
import ipywidgets
import random
import plotly
import io

In [2]:
# Conjunto de dados do Spotify
url = 'https://raw.githubusercontent.com/otaviofcoletti/VIZCOMP/main/SpotifyFeatures(1).csv'
df = pd.read_csv(url)

In [3]:
popular_music_rank = df.sort_values('popularity', ascending=False) 

## Pré-processamento

Para não selecionar artistas com apenas um single de sucesso (conhecidos como 'one hit'), trabalharemos com artistas que possuem 10 ou mais músicas populares para se ter como base artistas de sucesso mais consistentes.

In [4]:
vc = df['artist_name'].value_counts()  # contando quantas vezes cada artista aparece
df = df[df['artist_name'].isin(vc.index[vc.gt(10)])]  # selecionando apenas aqueles que aparecem 10 vezes ou mais

artist_count = df.groupby('artist_name')[['popularity']].count().sort_values(by='popularity', ascending=True)
artist_sum = df.groupby('artist_name')[['popularity']].sum().sort_values(by='popularity', ascending=True)

artist_rank = artist_sum.popularity / artist_count.popularity
artist_rank = artist_rank.sort_values(ascending=False)
artist_rank = pd.DataFrame(artist_rank).reset_index()


Abaixo vamos selecionar apenas as músicas dos artistas acima e conhecer quais são suas características mais importantes.

In [5]:
top10_artistas = list(artist_rank['artist_name'][0:50])
top10 = pd.DataFrame()

for i in top10_artistas:
  top10 = top10.append(df[df['artist_name'].str.match(i)],ignore_index=True)

As funções abaixo irão nos ajudar a plotar os gráficos que indicarão como são em média as músicas dos artistas mais populares do Spotify.

In [6]:
def artist_rank_calculator(feature : str):

  artist_count = top10.groupby('artist_name')[[feature]].count().sort_values(by=feature, ascending=True)
  artist_sum = top10.groupby('artist_name')[[feature]].sum().sort_values(by=feature, ascending=True)
  artist_rank = artist_sum[feature] / artist_count[feature]
  artist_rank = artist_rank.sort_values(ascending=False)
  artist_rank = pd.DataFrame(artist_rank).reset_index()

  return artist_rank

In [7]:
def plot_artist_rank (number_of_artist=1,characterists='valence'):

  ar = artist_rank_calculator(characterists)
  feature = characterists
  title = characterists
  fig = px.bar(ar[0:number_of_artist].sort_values(by=feature, ascending=True), y="artist_name", x=feature, color=feature, title="Artistas mais populares e seu " + title + ' médio',orientation='h',width=900, height=700)
  fig.update_layout(xaxis_range=[0,1])

  fig.update_layout(
      font_family="Helvetica",
      font_color="black",
      title_font_family="Helvetica",
      title_font_color="black",
      legend_title_font_color="black"
  )

  fig.update_xaxes(title_font_family="Helvetica")
  fig.show()

In [8]:
lista_features = ['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness']

In [9]:
dd = widgets.Dropdown(
    options=lista_features,
    value='valence',
    description='Atributo da Música',
    disabled=False,
    layout=Layout(width='30%', height='big')
)

## Gráfico de barras


In [10]:
ipywidgets.interactive(plot_artist_rank, number_of_artist=ipywidgets.IntSlider(min=1,max=50,step=1,value=10,description='N° artistas'), characterists=dd)

interactive(children=(IntSlider(value=10, description='N° artistas', max=50, min=1), Dropdown(description='Atr…

## Radviz

Temos aqui quatro eixos nos quais as músicas estarão dispostas para análise --- *energy, acousticness, danceability* e *popularity.*

In [11]:
def radviz_vs (genero_1 : str,genero_2 : str,genero_3 : str, number_of_music=150):
  
  fig = plt.figure( )
  plt.figure(figsize=(15,10))
  atributes = df[['acousticness','energy','danceability','popularity', 'genre']]
  pd.plotting.radviz(atributes[atributes['genre'].isin([genero_1])][:number_of_music], 'genre',color=['blue'])
  pd.plotting.radviz(atributes[atributes['genre'].isin([genero_2])][:number_of_music], 'genre',color=['red'])
  pd.plotting.radviz(atributes[atributes['genre'].isin([genero_3])][:number_of_music], 'genre',color=['green'])
  plt.title("Disposição das músicas dos gêneros ")
  plt.legend( loc='upper right', fontsize='larger' )

In [12]:
lista_generos = ['Genero 1','Genero 2','Genero 3']

In [13]:
genres=['Dance',
 'Rap',
 'Hip-Hop',
 'Pop',
 'Reggaeton',
 "Children's Music",
 'Folk',
 'Rock',
 'Indie',
 'Electronic',
 'Alternative',
 'Country',
 'Soul',
 'Blues',
 'Jazz',
 'Reggae',
 'R&B',
 'World',
 'Soundtrack',
 'Movie',
 'Classical',
 'Ska',
 'Anime',
 'Opera',
 'Comedy',
 'A Capella']

In [14]:
random.seed(10)
dd = {description: widgets.Dropdown(
    options=genres,
    value=genres[random.randint(0,len(genres)-1)],
    description=description,
    disabled=False,
) for description in lista_generos }

Escolha o gênero musical e número de músicas a comparar:

In [15]:
ipywidgets.interact(radviz_vs, genero_1=dd['Genero 1'] ,genero_2=dd['Genero 2'] ,genero_3=dd['Genero 3'],number_of_music=ipywidgets.IntSlider(min=0, max=1000,step=25,value=150,description="N° músicas"))

interactive(children=(Dropdown(description='Genero 1', index=18, options=('Dance', 'Rap', 'Hip-Hop', 'Pop', 'R…

<function __main__.radviz_vs(genero_1: str, genero_2: str, genero_3: str, number_of_music=150)>